In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
import rioxarray as rxr
from myfunc import timer
from myfunc import DirMan
from myfunc import load_and_flatten_data
import config

resolution     = config.resolution
region         = config.region
data_path      = config.data_path
post_data_path = config.post_data_path
shp_path       = config.shp_path
fig_path       = config.fig_path

dir_man = DirMan(data_path)
dir_man.enter()

os.makedirs(f'{data_path}/csv', exist_ok=True)

area_sum = 19.925158940748766


Entered /tera11/zhwei/students/Xionghui/data/run/0p1/


In [4]:
df = pd.DataFrame()

with nc.Dataset('Sbedrock.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    # ('mask123', 'Band1'),
    # ('Sbedrock', 'Sbedrock'),
    # ('Sr', 'Sr'),
    # ('Ssoil', 'Band1'),
    ('Area','area'),

    ('DTB','Band1'),
    ('IGBP','LC',0),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  

    # print(f"File: {file}, Variable: {variable_name}, Index: {index}")
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)


df = df.dropna()
df = df[df['IGBP'] < 10]

# df = df[df['Sbedrock'] > 0]
# df = df[df['mask123'] == 1]

# print((df.groupby('D_FM_median')['Area'].sum()/1e12)/16.092963536896)
area_sum = df['Area'].sum()/1e12
area_shallow_bedrock_sum = df[df['DTB'] <= 150]['Area'].sum()/1e12


print(area_sum)
print(area_shallow_bedrock_sum)
print(area_shallow_bedrock_sum/area_sum)



# df.drop(labels='mask123',axis=1,inplace=True)
# df.drop(labels='Area',axis=1,inplace=True)

# with open('csv/Global_S.csv','w') as f:
#     df.to_csv(f)
df

63.746672164864
34.464535150592
0.54064838179252


,lat,lon,Area,DTB,IGBP
1228724,-55.85,-67.55,69412200.0,49.948780,8
1228728,-55.85,-67.15,69412200.0,75.096161,9
1232326,-55.75,-67.35,69590688.0,50.141003,9
1235919,-55.65,-68.05,69768968.0,50.000141,9
1235926,-55.65,-67.35,69768968.0,124.904091,8
...,...,...,...,...,...
5971569,75.85,96.95,30229888.0,100.000000,9
5971624,75.85,102.45,30229888.0,100.000000,9
5978921,76.05,112.15,29811140.0,2221.104004,9
6175522,81.55,-27.75,18171250.0,0.000000,8
